In [4]:
# Import required libraries
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
!gdown 19xU3kUyivRWGQzYXnpwlIjQqdfkD56v4

Downloading...
From: https://drive.google.com/uc?id=19xU3kUyivRWGQzYXnpwlIjQqdfkD56v4
To: /content/twitter_data.csv
100% 10.3M/10.3M [00:00<00:00, 56.1MB/s]


In [5]:
data = pd.read_csv('/content/twitter_data.csv')

In [9]:
data = data.drop(columns=['id','user'], axis=0)

In [10]:
data

,sentiment,tweet
0,Positive,im getting on borderlands and i will murder yo...
1,Positive,I am coming to the borders and I will kill you...
2,Positive,im getting on borderlands and i will kill you ...
3,Positive,im coming on borderlands and i will murder you...
4,Positive,im getting on borderlands 2 and i will murder ...
...,...,...
74677,Positive,Just realized that the Windows partition of my...
74678,Positive,Just realized that my Mac window partition is ...
74679,Positive,Just realized the windows partition of my Mac ...
74680,Positive,Just realized between the windows partition of...


In [7]:
# Load the pre-trained model
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:
# Prepare the data
class SentimentDataset(data):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Tokenize the text using the loaded tokenizer and convert it to PyTorch tensors
        encoded_text = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        return encoded_text, label

ValueError: Buffer has wrong number of dimensions (expected 1, got 3)

In [ ]:
# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.2)

In [ ]:
# Create the datasets and dataloaders
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64)

In [ ]:
# Define the optimizer and learning rate scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

In [ ]:
# Train the model
model.train()
for epoch in range(5):
    for batch in train_dataloader:
        inputs, labels = batch
        # Clear the gradients from the previous batch
        optimizer.zero_grad()
        # Forward pass through the model
        outputs = model(**inputs, labels=labels)
        # Compute the loss
        loss = outputs.loss
        # Backward pass through the loss and compute gradients
        loss.backward()
        # Update the model parameters
        optimizer.step()
        # Update the learning rate
        scheduler.step()

In [ ]:
# Evaluate the model
model.eval()
num_correct = 0
num_total = 0
with torch.no_grad():
    for batch in val_dataloader:
        inputs, labels = batch
        # Forward pass through the model
        outputs = model(**inputs)
        # Compute the predicted labels
        predicted_labels = torch.argmax(outputs.logits, axis=1)
        # Compute the number of correct and total predictions
        num_correct += torch.sum(predicted_labels == labels)
        num_total += len(labels)
# Compute the validation accuracy
accuracy = float(num_correct) / float(num_total)
print("Validation accuracy: {:.2f}%".format(accuracy * 100))